In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

In [ ]:
from numpy.fft import fft, ifft, fftfreq
from scipy.interpolate import interp1d

def random_profile(t0, tmax, nt, fmax):
    freqs = fftfreq(nt)
    tff = np.zeros((nt,), dtype=np.complex)
    ncomps = len(freqs[np.abs(freqs)<fmax])
    tff[np.abs(freqs)<fmax] = np.random.normal(size=(ncomps,)) + np.random.normal(size=(ncomps,))*1j
    profile = ifft(tff).real
    profile = (profile - profile.min()) / (profile.max() - profile.min())
    t = np.linspace(t0, tmax, nt)
    return interp1d(t, profile)

In [ ]:
from flapjack import *
fj = Flapjack(url_base='3.128.232.8:8000')
#fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study):
    fj.delete('study', study.id[0])

### Constitutive gene

In [ ]:
dna = fj.get('dna', name='pREC1')
if len(dna)==0:
    dna = fj.create('dna', name='Const2')
vector = fj.get('vector', name='pREC1')    
if len(vector)==0:
    vector = fj.create('vector', name='Const2', dnas=dna.id)
    
const = GeneticNetwork(vector=vector.id[0])

In [ ]:
sfp1 = fj.get('signal', name='SFP0')
sfp1 = Reporter(name='CFP', degradation_rate=0, init_concentration=0, signal_id=sfp1.id[0])

const.add_reporter(sfp1)

In [ ]:
#def profile(t):
#    return 1 - gompertz_growth_rate(t, 0.01, 1, 1, 4)
profile = random_profile(t0=0, tmax=24, nt=100, fmax=0.025)
const.add_operator(Source(output=sfp1, rate=1, profile=profile))

In [ ]:
t = np.linspace(0, 24, 100)
plt.plot(t, profile(t))

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)
    
od = fj.get('signal', name='OD')
const_study = fj.get('study', name=['Context effects'])
vector = fj.get('vector', name='pAAA')
print(vector)
media = fj.get('media', name='M9-glucosa')
print(media)
strain = fj.get('strain', name='Top10')
print(strain)

metab = SimulatedMetabolism(biomass, growth_rate)
#metab = DataMetabolism(fj, media, strain, vector, od)

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

sample = Sample(circuit=const, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])

In [ ]:
print(sample.biomass)

In [ ]:
biomass_signal = fj.get('signal', name='OD')
print(biomass_signal)
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.24,
              name='Loica constitutive expression',
              description='Simulated constitutive gene generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Biomass'].plot(x='Time', y='Measurement', style='.', ax=ax)

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Testing')
assay.upload(fj, study.id[0])

In [ ]:
vector = fj.get('vector', name='pREC1')
media = fj.get('media', name='Simulated media')
strain = fj.get('strain', name='Simulated strain')
cfp = fj.get('signal', name='SFP0')
biomass_signal = fj.get('signal', name='SOD')

print(biomass_signal)

source = Source(None, 0) #const.operators[0]
source.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=cfp.id,
    biomass_signal=biomass_signal.id,
    fmax=0.1,
    epsilon=0
)

In [ ]:
J = source.res.jac
np.linalg.inv(J.T.dot(J))

In [ ]:
print('Protein degradation rate = ', source.gamma)
t = np.linspace(1, 23, 100)
plt.plot(t, source.profile(t))
plt.plot(t, profile(t)*1e6*0.5)
#plt.plot(t, 1 - gompertz_growth_rate(t, 0.01, 1, 1, 4))
#plt.plot(t, 1000 - 500 * gompertz_growth_rate(t, 0.01, 1, 1, 4))
plt.legend(['Characterization', 'True'])

### Receiver

Create a genetic network and associate it with a Flapjack vector:

In [ ]:
dna = fj.get('dna', name='Rec10')
if len(dna)==0:
    dna = fj.create('dna', name='Rec10')
vector = fj.get('vector', name='Rec10')    
if len(vector)==0:
    vector = fj.create('vector', name='Rec10', dnas=dna.id)
    
rec = GeneticNetwork(vector=vector.id[0])

Create a reporter and associate it with a Flapjack signal so we can record the behaviour of the circuit:

In [ ]:
cfp = fj.get('signal', name='CFP')
sfp1 = Reporter(name='CFP', degradation_rate=0, init_concentration=1, signal_id=cfp.id[0])

rec.add_reporter(sfp1)

Create and add a receiver operator to the circuit, linking it to an AHL supplement:

In [ ]:
ahl = Supplement(name='AHL')
def sin_profile(t):
    return 1 - np.cos(2 * np.pi * t / 12)
def mu_profile(t):
    return 1 - gompertz_growth_rate(t, 0.01, 1, 1, 4)
rec_profile = random_profile(t0=0, tmax=24, nt=100, fmax=0.05)
rec.add_operator(Receiver(inducer=ahl, output=sfp1, a=0, b=1, K=1, n=2, profile=rec_profile))

Now we have constructed the circuit we need to run an assay containing some samples. The sample is driven by a metabolism which defines the dynamics of growth and gene expression profiles:

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)
    
metab = SimulatedMetabolism(biomass, growth_rate)

Next we create a set of samples associated to Flapjack media and strain, and containing our AHL at different concentrations

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

# Create list of samples    
samples = []
for conc in np.logspace(-6, 6, 12):
    sample = Sample(circuit=rec, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

Now we can create and run the assay:

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica receiver',
              description='Simulated receiver generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

Plot the results:

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].groupby('Sample').plot(x='Time', y='Measurement', style='.', ax=ax)
plt.yscale('log')
len(m)

Upload the simulated data to flapjack

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])

In [ ]:
#vector = fj.get('vector', name='pREC')
#media = fj.get('media', name='Simulated media')
#strain = fj.get('strain', name='Simulated strain')
#signal = fj.get('signal', name='CFP')
#biomass_signal = fj.get('signal', name='OD')
#print(biomass_signal)

signal = fj.get('signal', name='CFP')
vector = fj.get('vector', name='Rec10')
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
cfp = fj.get('signal', name='CFP')
biomass_signal = fj.get('signal', name='OD')
analyte = fj.get('chemical', name='AHL')

char_receiver = Receiver(None, None, 0,0,0,0)
char_receiver.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=signal.id,
    biomass_signal=biomass_signal.id,
    fmax=0.1,
    epsilon=0
)

In [ ]:
plt.plot(t, rec_profile(t))

In [ ]:
t = np.linspace(0, 24, 100)
print('a = ', char_receiver.a)
print('b = ', char_receiver.b)
print('K = ', char_receiver.K)
print('n = ', char_receiver.n)

plt.plot(t, char_receiver.profile(t))
plt.plot(t, rec_profile(t))
plt.legend(['Characterization', 'True'])

In [ ]:
J = char_receiver.res.jac
C = np.log(J.T.dot(J))
np.diag(np.linalg.inv(C))

### Inverter

In [ ]:
dna = fj.get('dna', name='Inv3')
if len(dna)==0:
    dna = fj.create('dna', name='Inv3')
vector = fj.get('vector', name='Inv3')    
if len(vector)==0:
    vector = fj.create('vector', name='Inv3', dnas=dna.id)
    
inv = GeneticNetwork(vector=vector.id[0])

In [ ]:
def mu_profile(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)
inv_profile = random_profile(t0=0, tmax=24, nt=100, fmax=0.05)

In [ ]:
ahl = Supplement(name='AHL')
laci = Regulator('LacI', degradation_rate=1)
rec = Receiver(inducer=ahl, output=laci, a=0, b=1e6, K=1, n=2, profile=rec_profile)
inv.add_operator(rec)
inv.add_regulator(laci)

In [ ]:
cfp = fj.get('signal', name='CFP')
cfp = Reporter('CFP', signal_id=cfp.id[0])
inv.add_reporter(cfp)
inv.add_operator(Not(input=laci, output=cfp, a=100, b=0, K=1, n=2, profile=inv_profile))

In [ ]:
# Create list of samples    
samples = []
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
for conc in np.logspace(-6, 6, 12):
    sample = Sample(circuit=inv, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica inverter',
              description='Simulated inverter generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])

In [ ]:
signal = fj.get('signal', name='CFP')
receiver = fj.get('vector', name='Rec3')
inverter = fj.get('vector', name='Inv3')
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
biomass_signal = fj.get('signal', name='OD')
analyte = fj.get('chemical', name='AHL')

char_not = Not(input=None, output=None, a=100, b=0, K=1, n=2)
char_not.characterize(
    fj,
    receiver = receiver.id,
    inverter=inverter.id,
    media=media.id,
    strain=strain.id,
    signal=signal.id,
    biomass_signal=biomass_signal.id,
    analyte=analyte.id,
    fmax=0.1,
    epsilon=0
)

In [ ]:
t = np.linspace(0, 24, 100)
plt.plot(t, char_not.profile(t))
plt.plot(t, inv_profile(t)*100)
#plt.plot(t, rec_profile(t)*100)
#plt.plot(t, not_.biomass*500)
#plt.ylim([0,100])

print(char_not.a, char_not.b, char_not.K, char_not.n, char_not.gamma)

In [ ]:
t = np.linspace(0, 24, 100)
plt.plot(t, char_receiver.profile(t))
plt.plot(t, rec_profile(t))

In [ ]:
meas_true = assay.measurements

inv_model = GeneticNetwork()
ahl = Supplement(name='AHL')
laci = Regulator('LacI', degradation_rate=1)
rec = Receiver(inducer=ahl, output=laci, 
               a=0, 
               b=char_receiver.a, 
               K=char_receiver.K, 
               n=char_receiver.n, 
               profile=char_receiver.profile)
inv_model.add_operator(rec)
inv_model.add_regulator(laci)

cfp = fj.get('signal', name='CFP')
cfp = Reporter('CFP', signal_id=cfp.id[0])
inv_model.add_reporter(cfp)
inv_model.add_operator(Not(input=laci, output=cfp, 
                           a=char_not.a, 
                           b=char_not.b, 
                           K=char_not.K, 
                           n=char_not.n, 
                           profile=char_not.profile)
                      )

In [ ]:
# Create list of samples    
samples = []
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
for conc in np.logspace(-6, 6, 12):
    sample = Sample(circuit=inv_model, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

In [ ]:
biomass_signal = fj.get('signal', name='OD')
model_assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica inverter',
              description='Simulated inverter generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
model_assay.run()

In [ ]:
m = model_assay.measurements
fig,ax = plt.subplots(1,1, figsize=(12,12))
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='+', ax=ax)

len(meas_true)
m = meas_true
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='r.', ax=ax)

#plt.xlim([0,5])
#plt.ylim([0,1])

In [ ]:
len(assay.measurements)

### Repressilator

In [ ]:
dna = fj.get('dna', name='Rep')
if len(dna)==0:
    dna = fj.create('dna', name='Rep')
vector = fj.get('vector', name='Rep')    
if len(vector)==0:
    vector = fj.create('vector', name='Rep', dnas=dna.id)
    
rep = GeneticNetwork(vector=vector.id[0])

In [ ]:
laci = Regulator(name='LacI', degradation_rate=1, init_concentration=5)
tetr = Regulator(name='TetR', degradation_rate=1)
ci = Regulator(name='cI', degradation_rate=1)
rep.add_regulator(laci)
rep.add_regulator(tetr)
rep.add_regulator(ci)

cfp = fj.get('signal', name='CFP')
yfp = fj.get('signal', name='YFP')
rfp = fj.get('signal', name='RFP')

sfp1 = Reporter(name='CFP', degradation_rate=1, signal_id=cfp.id[0])
rep.add_reporter(sfp1)
sfp2 = Reporter(name='YFP', degradation_rate=1, signal_id=yfp.id[0])
rep.add_reporter(sfp2)
sfp3 = Reporter(name='RFP', degradation_rate=1, signal_id=rfp.id[0])
rep.add_reporter(sfp3)

rep.add_operator(Not(input=ci, output=laci, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=laci, output=tetr, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=tetr, output=ci, a=100, b=0, K=1, n=2))

rep.add_operator(Not(input=ci, output=sfp1, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=laci, output=sfp2, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=tetr, output=sfp3, a=100, b=0, K=1, n=2))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

biomass_signal = fj.get('signal', name='OD')

sample = Sample(circuit=rep, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0]
               )
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica repressilator',
              description='Simulated repressilator generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
assay.measurements

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='YFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='RFP'].plot(x='Time', y='Measurement', ax=ax)

Upload simulated data to flapjack

In [ ]:
assay.upload(fj, study.id[0])

### Toggle switch

In [ ]:
dna = fj.create('dna', name='Toggle')
vector = fj.create('vector', name='Toggle', dnas=dna.id)
tog = GeneticNetwork(vector=vector.id[0])

laci = Regulator(name='LacI', degradation_rate=1, init_concentration=0.1)
ci = Regulator(name='cI', degradation_rate=1)
tog.add_regulator(laci)
tog.add_regulator(ci)

cfp = fj.get('signal', name='CFP')
yfp = fj.get('signal', name='YFP')
sfp1 = Reporter(name='CFP', degradation_rate=1, signal_id=cfp.id[0])
tog.add_reporter(sfp1)
sfp2 = Reporter(name='YFP', degradation_rate=1, signal_id=yfp.id[0])
tog.add_reporter(sfp2)

tog.add_operator(Not(input=laci, output=ci, a=10, b=0, K=1, n=2))
tog.add_operator(Not(input=ci, output=laci, a=10, b=0, K=1, n=2))

tog.add_operator(Not(input=ci, output=sfp1, a=10, b=0, K=1, n=2))
tog.add_operator(Not(input=laci, output=sfp2, a=10, b=0, K=1, n=2))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

biomass_signal = fj.get('signal', name='OD')
sample = Sample(circuit=tog, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0]
               )
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica toggle',
              description='Simulated toggle switch generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='YFP'].plot(x='Time', y='Measurement', ax=ax)

Upload simulated data to flapjack

In [ ]:
assay.upload(fj, study.id[0])

### Nor gate

In [ ]:
dna = fj.get('dna', name='Nor')
if len(dna)==0:
    dna = fj.create('dna', name='Nor')
vector = fj.get('vector', name='Nor')    
if len(vector)==0:
    vector = fj.create('vector', name='Nor', dnas=dna.id)
    
nor = GeneticNetwork(vector=vector.id[0])

Create a reporter and associate it with a Flapjack signal so we can record the behaviour of the circuit:

In [ ]:
cfp = fj.get('signal', name='CFP')
sfp1 = Reporter(name='CFP', degradation_rate=0, signal_id=cfp.id[0])

nor.add_reporter(sfp1)

Create and add a receiver operator to the circuit, linking it to an AHL supplement and the receptor we just created:

In [ ]:
ahl1 = Supplement(name='AHL1')
ahl2 = Supplement(name='AHL2')
nor.add_operator(Nor(input=[ahl1, ahl2], output=sfp1, alpha=[0.0001,1,1,1], a=[100,100], b=[1,1], K=[1,1], n=[2,2]))

Now we have constructed the circuit we need to run an assay containing some samples. The sample is driven by a metabolism which defines the dynamics of growth and gene expression profiles:

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)
    
metab = SimulatedMetabolism(biomass, growth_rate)

Next we create a set of samples associated to Flapjack media and strain, and containing our AHL at different concentrations

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

# Create list of samples    
samples = []
for conc1 in np.logspace(-3, 3, 6):
    for conc2 in np.logspace(-3,3,6):
        sample = Sample(circuit=nor, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
        # Add AHL to samples at given concentration
        sample.add_supplement(ahl1, conc1)
        sample.add_supplement(ahl2, conc2)
        samples.append(sample)

Now we can create and run the assay:

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.25,
              name='Loica nor',
              description='Simulated nor generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

Plot the results:

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

Upload the simulated data to flapjack

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])